# Le notebook que yohan n'a pas fait

In [10]:
import pandas as pd
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
import os

In [11]:
def merge_vehicle_count(caracteristiques, vehicules):
    # Compter le nombre de véhicules par "Num_Acc" dans le DataFrame vehicules
    vehicules_count = vehicules.groupby("Accident_Id")["id_vehicule"].count().reset_index().rename(columns={'id_vehicule':'Nombre_vehicule'})
    
    # Fusionner le DataFrame caracteristiques avec le DataFrame vehicules_count en utilisant la colonne "Num_Acc"
    merged_df = caracteristiques.merge(vehicules_count, on='Accident_Id')
    
    return merged_df


def merge_user_count(caracteristiques,usagers):
    # Compter le nombre d'usagers par "Accident_Id" et par sexe dans le DataFrame usagers
    usagers_count = usagers.groupby(["Accident_Id", "sexe"])["id_vehicule"].count().unstack(fill_value=0)
    usagers_count.rename(columns={1: 'Nombre_Hommes', 2: 'Nombre_Femmes'}, inplace=True)
    
    # Fusionner le DataFrame caracteristiques avec usagers_count en utilisant la colonne "Num_Acc"
    merged_df = caracteristiques.merge(usagers_count, on='Accident_Id')
    
    return merged_df

def replace_with_mapper(df, col, mapper):
    df[col] = df[col].replace(mapper)
    
    return df

In [12]:
years = ['19', '20', '21', '22']
files_name = ["caracteristiques-20", "lieux-20", "usagers-20", "vehicules-20"]
if not os.path.exists('./Dataset/all_years'):
    os.mkdir('./Dataset/all_years/', 0o666) 
for file_name in files_name:
    result = []
    print(f"\n///////////////{file_name}///////////////")
    for year in years:
        file_path = f'./Dataset/{year}/{file_name}{year}.csv'
        sep = ';'
        df = pd.read_csv(file_path, sep=sep, low_memory=False)
        # if 'id_usager' in df.columns:
        #     df = df.drop(columns=['id_usager'])
        result.append(df)
        print(df.columns.tolist())
    combined_df = pd.concat(result, ignore_index=True)
    combined_df.to_csv(f'./Dataset/all_years/{file_name}{years[0]}-{years[-1]}.csv', index=False)


///////////////caracteristiques-20///////////////
['Accident_Id', 'jour', 'mois', 'an', 'hrmn', 'lum', 'dep', 'com', 'agg', 'int', 'atm', 'col', 'adr', 'lat', 'long']
['Accident_Id', 'jour', 'mois', 'an', 'hrmn', 'lum', 'dep', 'com', 'agg', 'int', 'atm', 'col', 'adr', 'lat', 'long']
['Accident_Id', 'jour', 'mois', 'an', 'hrmn', 'lum', 'dep', 'com', 'agg', 'int', 'atm', 'col', 'adr', 'lat', 'long']
['Accident_Id', 'jour', 'mois', 'an', 'hrmn', 'lum', 'dep', 'com', 'agg', 'int', 'atm', 'col', 'adr', 'lat', 'long']

///////////////lieux-20///////////////
['Accident_Id', 'catr', 'voie', 'v1', 'v2', 'circ', 'nbv', 'vosp', 'prof', 'pr', 'pr1', 'plan', 'lartpc', 'larrout', 'surf', 'infra', 'situ', 'vma']
['Accident_Id', 'catr', 'voie', 'v1', 'v2', 'circ', 'nbv', 'vosp', 'prof', 'pr', 'pr1', 'plan', 'lartpc', 'larrout', 'surf', 'infra', 'situ', 'vma']
['Accident_Id', 'catr', 'voie', 'v1', 'v2', 'circ', 'nbv', 'vosp', 'prof', 'pr', 'pr1', 'plan', 'lartpc', 'larrout', 'surf', 'infra', 'situ', '

In [13]:
caracteristiques = pd.read_csv('./Dataset/all_years/caracteristiques-2019-22.csv')
# Remplacer les numéros des mois par leurs noms
month_mapper = {
    1:'Janvier',
    2:'Février',
    3:'Mars',
    4:'Avril',
    5:'Mai',
    6:'Juin',
    7:'Juillet',
    8:'Août',
    9:'Septembre',
    10:'Octobre',
    11:'Novembre',
    12:'Décembre'
}
caracteristiques.mois = caracteristiques.mois.map(month_mapper, na_action=None)

caracteristiques.head(10)

,Accident_Id,jour,mois,an,hrmn,lum,dep,com,agg,int,atm,col,adr,lat,long
0,201900000001,30,Novembre,2019,01:30,4,93,93053,1,1,1,2,AUTOROUTE A3,"48,8962100","2,4701200"
1,201900000002,30,Novembre,2019,02:50,3,93,93066,1,1,1,6,AUTOROUTE A1,"48,9307000","2,3688000"
2,201900000003,28,Novembre,2019,15:15,1,92,92036,1,1,1,4,AUTOROUTE A86,"48,9358718","2,3191744"
3,201900000004,30,Novembre,2019,20:20,5,94,94069,1,1,1,4,A4,"48,8173295","2,4281502"
4,201900000005,30,Novembre,2019,04:00,3,94,94028,1,1,1,2,A86 INT,"48,7763620","2,4332540"
5,201900000006,28,Novembre,2019,14:03,1,87,87085,1,3,2,3,FOCH (RUE DU MARECHAL),"45,8469400","1,2338600"
6,201900000007,28,Novembre,2019,07:27,2,69,69092,2,1,8,2,BEAUJOLAIS (avenue du),"46,0006600","4,7059900"
7,201900000008,29,Novembre,2019,16:40,2,38,38544,2,1,1,6,VICTOR HUGO(RUE),"45,5221900","4,8748000"
8,201900000009,28,Novembre,2019,18:00,3,34,34129,1,1,1,5,PALAVAS D986 (ROUTE DE),"43,5644580","3,8943420"
9,201900000010,28,Novembre,2019,14:50,1,13,13206,2,1,1,2,LOUIS SALVATOR (BOULEVARD),"43,2911000","5,3818900"


In [15]:
caracteristiques = pd.read_csv('./Dataset/all_years/caracteristiques-2019-22.csv')

caracteristiques = replace_with_mapper(caracteristiques, 'lum', {
    1 : 'Plein jour',
    2 : 'Crépuscule ou aube',
    3 : 'Nuit sans éclairage public',
    4 : 'Nuit avec éclairage public non allumé',
    5 : 'Nuit avec éclairage public allumé'
})
caracteristiques = replace_with_mapper(caracteristiques, 'agg', {
    1 : 'Hors agglomération',
    2 : 'En agglomération'
})
caracteristiques = replace_with_mapper(caracteristiques, 'int', {
    1 : 'Hors intersection',
    2 : 'Intersection en X',
    3 : 'Intersection en T',
    4 : 'Intersection en Y',
    5 : 'Intersection à plus de 4 branches',
    6 : 'Giratoire',
    7 : 'Place',
    8 : 'Passage à niveau',
    9 : 'Autre intersection'
})
caracteristiques = replace_with_mapper(caracteristiques, 'atm', {
    -1 :'Non renseigné',
    1 : 'Normale',
    2 : 'Pluie légère',
    3 : 'Pluie forte',
    4 : 'Neige - grêle',
    5 : 'Brouillard - fumée',
    6 : 'Vent fort - tempête',
    7 : 'Temps éblouissant',
    8 : 'Temps couvert',
    9 : 'Autre'
})
caracteristiques = replace_with_mapper(caracteristiques, 'col', {
    -1 :'Non renseigné',
    1 : 'Deux véhicules - frontale',
    2 : 'Deux véhicules – par l’arrière',
    3 : 'Deux véhicules – par le coté',
    4 : 'Trois véhicules et plus – en chaîne',
    5 : 'Trois véhicules et plus - collisions multiples',
    6 : 'Autre collision',
    7 : 'Sans collision'
})


caracteristiques

,Accident_Id,jour,mois,an,hrmn,lum,dep,com,agg,int,atm,col,adr,lat,long
0,201900000001,30,11,2019,01:30,Nuit avec éclairage public non allumé,93,93053,1,1,1,2,AUTOROUTE A3,"48,8962100","2,4701200"
1,201900000002,30,11,2019,02:50,Nuit sans éclairage public,93,93066,1,1,1,6,AUTOROUTE A1,"48,9307000","2,3688000"
2,201900000003,28,11,2019,15:15,Plein jour,92,92036,1,1,1,4,AUTOROUTE A86,"48,9358718","2,3191744"
3,201900000004,30,11,2019,20:20,Nuit avec éclairage public allumé,94,94069,1,1,1,4,A4,"48,8173295","2,4281502"
4,201900000005,30,11,2019,04:00,Nuit sans éclairage public,94,94028,1,1,1,2,A86 INT,"48,7763620","2,4332540"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218399,202200055298,1,1,2022,03:50,Nuit sans éclairage public,2B,2B293,1,1,1,6,D71,"42,3101650000","9,4785830000"
218400,202200055299,1,1,2022,07:20,Nuit sans éclairage public,84,84074,1,1,1,6,D973,"43,7531640000","5,2244760000"
218401,202200055300,1,1,2022,04:27,Nuit sans éclairage public,74,74001,1,1,9,6,D22,"46,2825320000","6,7328060000"
218402,202200055301,1,1,2022,08:40,Plein jour,81,81099,1,3,1,3,Chemin Toulze,"43,9272650000","1,9156370000"
